In [1]:
!pip install --upgrade pip
!pip install python-binance

# Option valuation model based on historical prices

In [1]:
import importlib.util
import os
google_colab_spec = importlib.util.find_spec("google") and importlib.util.find_spec("google.colab")
if google_colab_spec:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    path = '/content/drive/MyDrive/Colab Notebooks/Ethereum Prices'
else:
    path = f'{os.environ["HOME"]}/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices'
    
os.environ['CACHE_PATH'] = path
os.environ['DISABLE_CACHE'] = 'FALSE'

from datetime import timezone

import optionspricing
import binanceprices

COUNT_YEARS = 10
BINANCE_DATETIME_FORMAT = "%Y-%m-%d %H-%M-%S"
BINANCE_SYMBOL = 'ETHUSDT'

prices_df = binanceprices.load_prices(path, BINANCE_SYMBOL, COUNT_YEARS)



loading 2014 ............
loading 2015 ............
loading 2016 ............
loading 2017 ............
loading 2018 ............
loading 2019 ............
loading 2020 ............
loading 2021 ............
loading 2022 ............
loading 2023 ............
loading 2024 
interrupting at 2024/01


## Option valuation model: input parameters here

In [2]:
target_period_hours = 1 * 24 + 6
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2021, 7)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = 'https://www.deribit.com/api/v2/public'

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)

puts = trading_model.puts
calls = trading_model.calls
target_expiry = trading_model.target_expiry
strikes = trading_model.strikes
current_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size)

In [3]:
#
index_put = strikes_universe_size - 3
index_call = strikes_universe_size + 3
#
cost, value = (option_chain_df.iloc[index_put]['put_ask'] + option_chain_df.iloc[index_call]['call_ask'], 
 option_chain_df.iloc[index_put]['value_put_pct'] + option_chain_df.iloc[index_call]['value_call_pct']
)
print(f'target expiry: {target_expiry.astimezone(timezone.utc)} ({remaining_hours} hours left)')
print(f'current price: {current_price}')
print(f'trading put {option_chain_df.iloc[index_put].name:.0f} and call {option_chain_df.iloc[index_call].name:.0f}')
print(f'cost: {cost:.3f} / value: {value:.3f}, benefit/cost = {value / cost:.1f}x')
print(f'($) cost: {cost * current_price:.2f} / value: {value * current_price:.2f}, average gain = {(value - cost) * current_price:.2f}')

option_chain_df

target expiry: 2024-01-27 08:00:00+00:00 (24 hours left)
current price: 2200.21
trading put 2125 and call 2275
cost: 0.003 / value: 0.008, benefit/cost = 2.8x
($) cost: 6.16 / value: 16.98, average gain = 10.82


,value_call,value_put,value_call_median,value_put_median,value_call_pct,call_ask,value_put_pct,put_ask
strike,,,,,,,,
2100.0,107.427638,5.443590,100.387122,0.000000,NaN,0.0525,0.002474,0.0010
2125.0,85.105246,8.121198,75.387122,0.000000,NaN,0.0440,0.003691,0.0015
2150.0,64.199165,12.215117,50.387122,0.000000,NaN,0.0290,0.005552,0.0029
2175.0,45.500949,18.516901,25.387122,0.000000,NaN,0.0175,0.008416,0.0050
2200.0,30.374269,28.390221,0.387122,0.000000,NaN,0.0090,0.012903,0.0085
2225.0,20.248164,43.264116,0.000000,24.612878,0.009203,0.0044,NaN,0.0200
2250.0,13.505781,61.521733,0.000000,49.612878,0.006138,0.0023,NaN,0.0300
2275.0,8.858014,81.873966,0.000000,74.612878,0.004026,0.0013,NaN,NaN
2300.0,5.705778,103.721730,0.000000,99.612878,0.002593,0.0007,NaN,NaN
